# Burrows-Wheeler Alignment

In this document, we will use `bwa` to align `fastq` formatted sequence data with a reference genome. Such alignments, along with annotations from the reference genome, form the basis for many analyses. An important use case if Copy Number Analysis, e.g. in QDNAseq workflow you will go through for the next step.

Like the paper exercise, **the questions in this document are for your own benefit.** They will help you better understand what it is you are doing, but they are **not** part of the graded hand-in on Canvas!

<img src="img/bwa.png" width="400"/>

<div class="alert alert-block alert-info"><b>Note:</b> The reference in the <i>ref_demo</i> directory only contains the sequence of chromosome 18.<br><br>It is smaller, and therefore faster to work with than the entire human genome, but is still big enough to let you put BWA into practice.</div>

We have installed the `bwa` package on the VM this Jupyter instance runs on. Run the next cell and have a look at its options.

In [ ]:
bwa

# Preparation

Copy the sequence files from the remote location to your home directory.

In [ ]:
mkdir ~/ref_demo
mkdir ~/seq_demo
cp /local/data/bsb_asa/student/ref_demo/Homo_sapiens.GRCh38.dna.chromosome.18.fa ~/ref_demo/
cp /local/data/bsb_asa/student/seq_demo/SW480_GTGAAA_L008_R1_001.fastq ~/seq_demo/

### Step 1: Indexing

Like with the paper exercise, BWA starts by preprocessing the reference sequence. The next cell contains the command to do so. Compare the contents of **`~/ref_demo/`** before and after running it.

1. What files does indexing create?
2. What information do they contain?

In [ ]:
bwa index ~/ref_demo/Homo_sapiens.GRCh38.dna.chromosome.18.fa
echo Done

### Step 2: Aligning Reads to your Index

Now that you have an indexed reference, we can use it to align some reads!

**`~/seq_demo/SW480_GTGAAA_L008_R1_001.fastq`** contains almost 7 million of them, sequenced from the genome of the SW480 cell line. Run the next cell to find their suffix array intervals.

1. What does **`> ~/seq_demo/SW480_demo.sai`** at the end of the command mean?
2. What do the -n, -q and -t parameters do? *(Hint: check* **`man bwa`** *for the manual, or* **`bwa aln`** *for help.)*

In [ ]:
bwa aln -n 2 -q 40 -t 4 ~/ref_demo/Homo_sapiens.GRCh38.dna.chromosome.18.fa ~/seq_demo/SW480_GTGAAA_L008_R1_001.fastq > ~/seq_demo/SW480_demo.sai
echo Done

### Step 3: Single Ended Alignment

Because we are using single ended reads (as opposed to paired end), the last step is generating actual alignments from the suffix array intervals using **`bwa samse`**.

1. How many inputs does this function have? Do you understand why?
2. Did you include this step in the workflow you drew on paper? Why (not)?
3. What is the difference between single vs. paired end reads? How is this accounted for in this alignment step?

In [ ]:
bwa samse ~/ref_demo/Homo_sapiens.GRCh38.dna.chromosome.18.fa ~/seq_demo/SW480_demo.sai ~/seq_demo/SW480_GTGAAA_L008_R1_001.fastq > ~/seq_demo/SW480_demo.sam
echo Done

### Step 4: Converting Results to a Binary format

The **`.sam`** file you've generated contains the original reads aligned against your reference, serialized as flat text. Given the size of the files you're dealing with, encoding it more efficiently makes downstream analysis substantially faster. The next command uses the **`samtools`** package to encode your alignments in a binary **`.bam`** format, which is used by *many* sequence analysis tools -- including the [QDNAseq](3.%20QDNAseq.ipynb) R library.

1. What do **`.sam`** and **`.bam`** stand for?
2. Why is the original reference still an input to this command?
3. How many bits do you need to encode each nucleotide?
4. How many bits does a regular ASCII character take in UTF-8 encoding?
5. Based on this, what compression rate would you expect?<br>*(Hint: confirm your suspicions with* **`ls -la ~/seq_demo`**<br>*The big number in the middle of each line is the file's size in bytes.)*

In [ ]:
rm ~/seq_demo/SW480_GTGAAA_L008_R1_001.fastq
samtools view -b -T ~/ref_demo/Homo_sapiens.GRCh38.dna.chromosome.18.fa ~/seq_demo/SW480_demo.sam > ~/seq_demo/SW480_demo.bam
echo Done

<div class="alert alert-block alert-info"><b>Congratulations!</b> If you've made it this far -- and feel like you have a good answer to all the questions -- you are ready to proceed with the analysis of mapped reads in the <a href="3.%20QDNAseq.ipynb">QDNAseq</a> notebook!</div>